In [1]:
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
import os
import pandas as pd
import numpy as np
import cv2

In [2]:
model_path = 'C:/Projects/ISL-to-text/INCLUDE 50/MP_Models/pose_landmarker_heavy.task'
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
options_pose = PoseLandmarkerOptions(base_options = BaseOptions(model_asset_path = model_path),running_mode=VisionRunningMode.IMAGE)
MARGIN = 10 
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54)

In [3]:
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions

options_hand = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path="C:\Projects\ISL-to-text\INCLUDE 50\MP_Models\hand_landmarker.task"),
    running_mode=VisionRunningMode.IMAGE)
    

In [4]:
def draw_landmarks_pose(img, result):
    if len(result.pose_landmarks)>0:
        landmarks = result.pose_landmarks[0]
        image = np.copy(img)
        x = []
        y = []
        for landmark in landmarks:
            x.append(landmark.x)
            y.append(landmark.y)
        lm_proto = landmark_pb2.NormalizedLandmarkList()
        lm_proto.landmark.extend([landmark_pb2.NormalizedLandmark(x= landmark.x,y= landmark.y, z=landmark.z) for landmark in landmarks])
        solutions.drawing_utils.draw_landmarks(image,lm_proto)
        return image,x,y
    else:
        return img,[],[]
    
def draw_landmarks_hand(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)
    x_coordinates = []
    y_coordinates = []
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
            ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            solutions.hands.HAND_CONNECTIONS,
            solutions.drawing_styles.get_default_hand_landmarks_style(),
            solutions.drawing_styles.get_default_hand_connections_style())
    
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        cv2.putText(annotated_image, f"{handedness[0].category_name}",
                    (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                    FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

    return annotated_image,x_coordinates,y_coordinates

In [16]:
vid = cv2.VideoCapture("C:\Projects\ISL Research\INCLUDE50\Places_1of4\Places/19. House\MVI_3350.MOV")

while vid.isOpened():
    suc,frame = vid.read()
    while suc:
        cv2.imwrite('C:/Projects/ISL-to-text/INCLUDE 50/MP_Models/frame.png',frame)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        h,w,_  = frame.shape
        with PoseLandmarker.create_from_options(options_pose) as landmarker:
            results = landmarker.detect(mp_image)
        frame,_,_ = draw_landmarks_pose(frame,results)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks[0]
            min_x = min([(landmark.x)*w for landmark in landmarks]) -130
            max_x = max([(landmark.x)*w for landmark in landmarks]) +130
            min_y = min([(landmark.y)*h for landmark in landmarks]) -130
            max_y = max([(landmark.y)*h for landmark in landmarks])  +130
        img = frame[int(min_y):int(max_y),int(min_x):int(max_x)]
        cv2.imwrite('C:/Projects/ISL-to-text/INCLUDE 50/MP_Models/crop.png',img)
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=np.array(img))
        with HandLandmarker.create_from_options(options_hand) as landmarker:
            landmarks = landmarker.detect(image)
        an_img,x,y = draw_landmarks_hand(img,landmarks)
        print(landmarks)
        cv2.imwrite('C:/Projects/ISL-to-text/INCLUDE 50/MP_Models/annotatedframe.png',an_img)
        suc,frame = vid.read()
    vid.release()
cv2.destroyAllWindows()

HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand

In [6]:
print(landmarks)

HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])


Format for getting coordinates from landmarks

landmarks.pose_landmarks[0][index].x

In [19]:
#Load Frames form still image folder
path = 'C:\Projects\ISL-to-text\Word Level ISL'
for file in os.listdir(path):
    frame = cv2.imread(os.path.join(path,file))
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    h,w,_  = frame.shape
    with PoseLandmarker.create_from_options(options_pose) as landmarker:
        results = landmarker.detect(mp_image)
    frame,_,_ = draw_landmarks_pose(frame,results)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks[0]
        min_x = min([(landmark.x)*w for landmark in landmarks]) -130
        max_x = max([(landmark.x)*w for landmark in landmarks]) +130
        min_y = min([(landmark.y)*h for landmark in landmarks]) -130
        max_y = max([(landmark.y)*h for landmark in landmarks])  +130
    img = frame[int(min_y):int(max_y),int(min_x):int(max_x)]
    cv2.imwrite('C:/Projects/ISL-to-text/INCLUDE 50/MP_Models/crop.png',img)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=np.array(img))
    with HandLandmarker.create_from_options(options_hand) as landmarker:
        landmarks = landmarker.detect(image)
    an_img,x,y = draw_landmarks_hand(img,landmarks)
    print(landmarks)
    if(len(landmarks.hand_landmarks)!=0):
        cv2.imwrite('C:/Projects/ISL-to-text/INCLUDE 50/MP_Models/lmed'+file+'.png',an_img)

HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
HandLandmarkerResult(handedness=[[Category(index=0, score=0.9860396385192871, display_name='Right', category_name='Right')]], hand_landmarks=[[NormalizedLandmark(x=0.32192665338516235, y=0.25674980878829956, z=2.170611708152137e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.36755692958831787, y=0.24025171995162964, z=-0.00048339739441871643, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.39813095331192017, y=0.210581973195076, z=0.0026395416352897882, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4209929406642914, y=0.18653522431850433, z=0.0020429014693945646, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4333515167236328, y=0.16470035910606384, z=0.0022793535608798265, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.3659185469150543, y=0.16840100288391113, z=0.018954027444124222, visibi

In [32]:
image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.imread('C:\Projects\ISL-to-text\Word Level ISL\ANGRY (5).jpg'))
with HandLandmarker.create_from_options(options_hand) as landmarker:
       landmarks = landmarker.detect(image)
print(landmarks.handedness[0][0].display_name)

Right


Check for which hand, pair with check for how many hands

Notes:
1. If one of the hands is undetected/random, that hand can take any position, use to augment data, use atleast 20 random undetected hand positions to crfeate 20x new entries.
2. Match the number and side of hands detected in each category and eleminate the odd ones.
3. Augment the data based on the open and closed palm simulation.
4. augment the data using spacial transformations like the venus paper.
5. fred data into embedding layer (returns 3 vectors of 42,1)
6. Throw into a Transformer Encoder without the positional engoding.
7. Use a transformer decoder with avalible dictionary.